In [118]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow_addons import losses
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix , classification_report


ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
Cust_data = pd.read_csv("customer_churn.csv")
Cust_data.head()

In [ ]:
Cust_data.drop('customerID',axis ='columns',inplace = True)
Cust_data.dtypes

In [ ]:
Cust_data.TotalCharges.values

In [ ]:
Cust_data[pd.to_numeric(Cust_data.TotalCharges,errors='coerce').isnull()]

In [ ]:
Cust_data.iloc[488].TotalCharges

In [ ]:
print(Cust_data[Cust_data.TotalCharges != ' '].shape)
Cust_revised = Cust_data[Cust_data.TotalCharges != ' ']
print(Cust_revised.shape)

In [ ]:
Cust_revised['TotalCharges'] = pd.to_numeric(Cust_revised.TotalCharges)

In [ ]:
Cust_revised['TotalCharges']

In [ ]:
Cust_wtenure_stay = Cust_revised[Cust_revised['Churn']=='No'].tenure
Cust_wtenure_left = Cust_revised[Cust_revised['Churn']=='Yes'].tenure
plt.hist([Cust_wtenure_left,Cust_wtenure_stay],label = ['Leaving_Customers','Staying_Customers'])
plt.legend()
plt.xlabel("No of years in tenure")
plt.ylabel("Number of customers")
plt.title("Churn status of customers and tenure status")

In [ ]:
def unique_value_categorical_columns(Cust_revised):    
    for column in Cust_revised:
        if Cust_revised[column].dtypes == 'object':
            print (f'{column} : {Cust_revised[column].unique()}')
unique_value_categorical_columns(Cust_revised)


In [ ]:
Cust_revised.replace('No phone service','No',inplace = True)
Cust_revised.replace('No internet service','No',inplace = True)


In [ ]:
unique_value_categorical_columns(Cust_revised)

In [ ]:
change_columns = ['DeviceProtection','Partner','Dependents','StreamingMovies','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'TechSupport','StreamingTV','PaperlessBilling','Churn']
for column in change_columns:
    Cust_revised[column].replace({'Yes':1,'No':0},inplace = True)

In [ ]:
for column in Cust_revised:
    print (f'{column} : {Cust_revised[column].unique()}')

In [ ]:
Cust_revised['gender'].replace({'Female':1,'Male':0},inplace=True)

In [ ]:
Cust_revised = pd.get_dummies(Cust_revised, columns = ['PaymentMethod','Contract','InternetService'])
Cust_revised

In [ ]:
columns_to_scale = ['tenure','MonthlyCharges','TotalCharges']

scaler = MinMaxScaler()
Cust_revised[columns_to_scale] = scaler.fit_transform(Cust_revised[columns_to_scale])

In [ ]:
Cust_revised.head()

In [ ]:
X = Cust_revised.drop('Churn',axis='columns')
y = Cust_revised['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=5)
print(X_train.shape)
print(X_test.shape)
X_train[:5]



In [119]:
def mynn(X_train,y_train,X_test,y_test,loss,weights):

    model = keras.Sequential([
        keras.layers.Dense(25, input_shape=(26,), activation='relu'),
        keras.layers.Dense(25, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')])

    model.compile(optimizer='adam',loss=loss,metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=20)
    else:
        model.fit(X_train, y_train, epochs=20,class_weight = weights)
    
    print(model.evaluate(X_test,y_test))
    yp = model.predict(X_test)
    yp = np.round(yp)
    
    print("Classificatoin report: \n",classification_report(y_test,yp))
    
    return yp

In [120]:
y_preds = mynn(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/20
259/259 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7385
Epoch 2/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4942 - accuracy: 0.7622
Epoch 3/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4854 - accuracy: 0.7651
Epoch 4/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4808 - accuracy: 0.7669
Epoch 5/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4770 - accuracy: 0.7692
Epoch 6/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4735 - accuracy: 0.7701
Epoch 7/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4692 - accuracy: 0.7754
Epoch 8/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4665 - accuracy: 0.7797
Epoch 9/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4626 - accuracy: 0.7824
Epoch 10/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4606 - accuracy: 0.7788

In [121]:
y_test.value_counts()

1    1033
0    1033
Name: Churn, dtype: int64

In [122]:
count_class_0,count_class_1 = Cust_revised.Churn.value_counts()
count_class_0,count_class_1
df_class_0 = Cust_revised[Cust_revised['Churn'] == 0]
df_class_1 = Cust_revised[Cust_revised['Churn'] == 1]

df_class_0.shape,df_class_1.shape

((5163, 27), (1869, 27))

In [123]:
'''y_pred = []
for value in yp:
    if value > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)'''

'y_pred = []\nfor value in yp:\n    if value > 0.5:\n        y_pred.append(1)\n    else:\n        y_pred.append(0)'

In [124]:
df_class_0_under_sample = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under_sample,df_class_1],axis =0)
df_test_under.shape
print('Random under-sampling')
print(df_test_under.Churn.value_counts())

Random under-sampling
1    1869
0    1869
Name: Churn, dtype: int64


In [125]:
X = df_test_under.drop('Churn',axis='columns')
y = df_test_under['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [126]:
y_preds = mynn(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/20
94/94 [==============================] - 0s 1ms/step - loss: 0.5898 - accuracy: 0.7194
Epoch 2/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4989 - accuracy: 0.7609
Epoch 3/20
94/94 [==============================] - 0s 3ms/step - loss: 0.4898 - accuracy: 0.7656
Epoch 4/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4810 - accuracy: 0.7709
Epoch 5/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4766 - accuracy: 0.7702
Epoch 6/20
94/94 [==============================] - 0s 3ms/step - loss: 0.4751 - accuracy: 0.7709
Epoch 7/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4706 - accuracy: 0.7756
Epoch 8/20
94/94 [==============================] - 0s 5ms/step - loss: 0.4683 - accuracy: 0.7729
Epoch 9/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4650 - accuracy: 0.7776
Epoch 10/20
94/94 [==============================] - 0s 2ms/step - loss: 0.4633 - accuracy: 0.7819
Epoch 11/20
94/94 [

Oversampling

In [127]:
count_class_0,count_class_1

(5163, 1869)

In [128]:
df_class_1_over = df_class_1.sample(count_class_0,replace = True)
df_test_over = pd.concat([df_class_0,df_class_1_over],axis = 0)
print("Random over-sampling \n",df_test_over.Churn.value_counts())

Random over-sampling 
 1    5163
0    5163
Name: Churn, dtype: int64


In [129]:
X = df_test_over.drop('Churn',axis='columns')
y = df_test_over['Churn']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify = y)
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [130]:
y_preds = mynn(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/20
259/259 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7308
Epoch 2/20
259/259 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.7599
Epoch 3/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4954 - accuracy: 0.7609
Epoch 4/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4909 - accuracy: 0.7643
Epoch 5/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4861 - accuracy: 0.7663
Epoch 6/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4839 - accuracy: 0.7668
Epoch 7/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4783 - accuracy: 0.7662
Epoch 8/20
259/259 [==============================] - 0s 1ms/step - loss: 0.4767 - accuracy: 0.7703
Epoch 9/20
259/259 [==============================] - 0s 2ms/step - loss: 0.4718 - accuracy: 0.7740
Epoch 10/20
259/259 [==============================] - ETA: 0s - loss: 0.4676 - accuracy: 0.77 - 0s 

SMOTE 